<a href="https://colab.research.google.com/github/JavZon/Machine_Learning/blob/main/Masala_LR_model_boyicha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

Datasetni chaqirib olamiz

In [2]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Datasetni 2 qismga bo'lamiz

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(df,test_size=0.2, random_state=42)

Datasetni housing ga yuklab olib median_house_value ni o'chirib yuboramiz

In [4]:
X_train = train_set.drop(columns='median_house_value')

y = train_set["median_house_value"].copy() # faqat labellarini saqlab oldik

X_num = X_train.drop(columns='ocean_proximity')

**Transformer** yasaymiz. Bu transformer har bir uydagi xonalar soni (rooms_per_household) va har bir uydagi odamlar sonini( population_per_household) aniqlab beradi

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      pass
    def fit(self, X, y=None):
      return self
    def transform(self, X):
      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
      population_per_household=X[:, population_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household]

Sonli ustunlar NaN larning o'rnini to'ldirish uchun **SimpleImputer** va datasetni -1 va 1 oralig'iga keltirish uchun **StandardScaler**ni chaqiramiz

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('custom_obj_adder', CustomTransformer()),
    ('std_scaler', StandardScaler())
])

prepared_num=numeric_pipeline.fit_transform(X_num)
prepared_num[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609])

**ColumnTransformer** nomli yangi classni chaqirib olamiz sklearn.compose ning ichidan.Bu class yakuniy full pipeline yaratish uchun ishlatiladi

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols=list(X_num.columns)
string_cols=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('numeric', numeric_pipeline, num_cols),
    ('string_cat',OneHotEncoder(), string_cols)
])

In [8]:
X_prepared=t=full_pipeline.fit_transform(X_train)

X_prepared[:3,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

In [28]:
y[:2]

14196    103000.0
8267     382100.0
Name: median_house_value, dtype: float64

In [9]:
X_train.index[:2]

Index([14196, 8267], dtype='int64')

# **Machine Learning** qismini boshlaymiz.

In [59]:
X_prepared[0,:]

array([ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
        0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ])

**Linear Regression** algorithmini chaqirib olamiz

In [55]:
from sklearn.linear_model import LinearRegression

LR_model= LinearRegression() # bu classdan object yaratib oldik va LR_model deb nom berdik

In [60]:
LR_model.fit(X_prepared,y)

LinearRegression()

In [70]:
trail_test=X_train.sample(10)
test_label=y.loc[trail_test.index]
trail_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
7379,-118.24,33.97,41.0,1182.0,346.0,1644.0,346.0,2.1473,<1H OCEAN
15324,-117.35,33.19,28.0,2823.0,476.0,1189.0,433.0,5.1733,NEAR OCEAN
8871,-118.41,34.06,43.0,2665.0,556.0,1015.0,506.0,4.1411,<1H OCEAN
3146,-118.51,35.16,7.0,4371.0,727.0,1932.0,654.0,4.6250,INLAND
15444,-117.26,33.19,2.0,2629.0,509.0,1044.0,522.0,4.2361,NEAR OCEAN
8436,-118.37,33.92,39.0,1073.0,206.0,556.0,204.0,4.8611,<1H OCEAN
14836,-117.10,32.68,42.0,2013.0,568.0,1920.0,557.0,2.0724,NEAR OCEAN
2287,-119.74,36.79,28.0,2857.0,619.0,1614.0,592.0,2.1573,INLAND
7561,-118.19,33.90,36.0,1073.0,271.0,1385.0,288.0,2.3214,<1H OCEAN
11902,-117.39,33.97,52.0,3307.0,553.0,1269.0,529.0,4.3176,INLAND


In [75]:
trail_test.shape

(10, 9)

In [71]:
test_label

7379     115100.0
15324    198100.0
8871     500001.0
3146     136800.0
15444    158500.0
8436     245600.0
14836    107600.0
2287      71400.0
7561     104800.0
11902    136200.0
Name: median_house_value, dtype: float64

In [72]:
test_prepared=full_pipeline.transform(trail_test)
test_prepared

array([[ 1.08545676e+00, -1.03522759e+00,  7.12940025e-01,
        -6.83529345e-01, -4.87488436e-01,  9.20976000e-02,
        -4.54272787e-01, -1.41236837e+00, -1.82623582e+00,
         2.72684523e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.59151046e+00, -1.44636686e+00, -2.73040861e-01,
        -3.40755467e-02, -2.57039357e-01, -2.84740602e-01,
        -2.92629710e-01,  1.38315670e+00,  1.24245149e+00,
        -3.57163004e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00],
       [ 9.88794818e-01, -9.87788446e-01,  8.64629392e-01,
        -9.66067473e-02, -1.15224539e-01, -4.28850155e-01,
        -1.56998162e-01,  4.29574093e-01,  3.65098716e-03,
        -1.49516973e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 9.31934852e-01, -4.07976658e-01, -1.86577921e+00,
         5.78571913e-01,  1.87904634e-01,  3.30623758e-01,
         1.17980867e-01,  8.76617908e-01,  1.

In [73]:
predicted_labels=LR_model.predict(test_prepared)

ValueError: X has 14 features, but LinearRegression is expecting 15 features as input.

In [54]:
predicted_labels

array([194063.57540465, 199848.69885622, 365857.6501014 , 249861.48692499,
        67157.77189079, 395495.39141857, 121899.60827313])

In [53]:
df2=pd.DataFrame({'Predicted_labels':predicted_labels, 'Real_labels':test_label})

ValueError: array length 7 does not match index length 10

In [47]:
df2['Difference']=df2['Predicted_labels']-df2['Real_labels']
df2

,Predicted_labels,Real_labels,Difference
16811,194063.575405,244000.0,-49936.424595
1545,199848.698856,350000.0,-150151.301144
6332,365857.650101,325000.0,40857.650101
107,249861.486925,259400.0,-9538.513075
3089,67157.771891,72500.0,-5342.228109
8315,395495.391419,414700.0,-19204.608581
3084,121899.608273,81200.0,40699.608273


In [ ]:
[1411,17444,16522,659,16498,11151,177,4421,3908,10362]